In [ ]:
# default_exp covariables

# modelos

> Herramientas de ajuste, validación y evaluación de modelos

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from criminologia_cdmx.etl import *
from criminologia_cdmx. covariables import *

## variable_independiente

In [ ]:
def variable_independiente(datos, columna_y, valor_y, fecha_inicio, fecha_fin, agregacion='colonias'):
    """ Regresa un DataFrame con la variable independicente agregada entre fecha_inicio y fecha_fin
        en las unidades requeridas.
        
        Args:
            datos (DataFrame): carpetas/victimas con ids espaciales y categorías de usuario
            columna_y (str): Nombre de la columna en donde vienen los incidentes de valor_y
            valor_y (str): delito o categoría a utilizar como Y
            fecha_inicio (str): fecha inicial para agregar delitos "d-m-Y"
            fecha_fin (str): fecha final para agregar delitos "d-m-Y"
            agregacion (str): colonias/cuadrantes. Eventualmente debe recibir 
             agregaciones arbitrarias
    """
    fecha_inicio = pd.to_datetime(fecha_inicio, dayfirst=True)
    fecha_fin = pd.to_datetime(fecha_fin, dayfirst=True)
    datos = datos.loc[datos['fecha_hechos'].between(fecha_inicio, fecha_fin)]
    datos = datos.loc[datos[columna_y] == valor_y]
    if agregacion == 'colonias':
        columna_agrega = 'colonia_cve'
        layer = 'colonias'
    elif agregacion == 'cuadrantes':
        columna_agrega = 'cuadrante_id'
        layer = 'cuadrantes'
    else:
        raise ValueError("unidades debe ser 'colonias' o 'cuadrantes'")
    datos = datos.groupby(columna_agrega).size()
    datos.name = valor_y
    unidades = gpd.read_file("datos/criminologia_capas.gpkg", layer=layer)
    datos = unidades[[columna_agrega]].merge(datos, on=columna_agrega, how='left').fillna(0)
    return datos   

In [ ]:
carpetas = get_carpetas_from_api(100000)
carpetas = agrega_ids_espaciales(carpetas)
fecha_inicio = carpetas.fecha_hechos.min().strftime("%d-%m.%Y")
fecha_fin = carpetas.fecha_hechos.max().strftime("%d-%m.%Y")
delito = 'ROBO A CASA HABITACION SIN VIOLENCIA'
Y = variable_independiente(carpetas, 'delito', delito, fecha_inicio, fecha_fin)
assert delito in Y.columns
Y = variable_independiente(carpetas, 'delito', delito, fecha_inicio, fecha_fin, 
                           agregacion='cuadrantes')
assert delito in Y.columns
Y

,cuadrante_id,ROBO A CASA HABITACION SIN VIOLENCIA
0,018,31.0
1,0110,28.0
2,014,40.0
3,015,34.0
4,018,31.0
...,...,...
842,043,15.0
843,046,12.0
844,045,15.0
845,038,10.0
